In [1]:
import meshcat
import meshcat.geometry as geom
import meshcat.transformations as tf
import numpy as np
import time
import jax.numpy as jnp
import jax
from jaxlie import SE2, SE3
import matplotlib.pyplot as plt
import math

In [2]:
########################################## Parameters ##########################################
_m1 = 0.1 #mass
_m2 = 0.1
_m3 = 0.1

_l1 = 5 #lenght
_l2 = 0.1
_l3 = 0.1

dt = 1e-3 #time step
_g = 9.81 #gravity

q = 0.35 #rads
q_dot = 0 #rads/s
T1 = 0

q1 = 0
q_dot = 0
T2 = 1


In [3]:
def lagrange(q, q_dot):
    """
    get q_ddot for euler
    """

    L1 = ((1/8) * _m1 * (_l1**2) * (q_dot**2)) - ((1/2) * _m1 *_g * _l1 * math.cos(q))
    
    q_ddot = ((4 * L1) / (_m1 * (_l1**2)) - (2 * (_g / _l1) * math.sin(q)) )

    return q_ddot

def euler(dt, q, q_dot, q_ddot):
    q_dot += (q_ddot * dt)
    q += (q_dot * dt)

    return q, q_dot


In [4]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [5]:

vis["base_pivot"].set_object(geom.Sphere(0.01))
link = vis["pendulum_link"]
link.set_object(geom.Box([0.01, 0.01, _l1]))
link.set_transform(tf.translation_matrix([0.0,0.0, (_l1/2)]))

In [ ]:
###### DO NOT RUN THIS #############################

#obj = vis["obj"]

#pivot_0 = obj["pivot_0"]
#link1 = pivot_0["link1"]

#pivot_1 = obj["pivot_1"]
#link2 = pivot_1["link2"]

#pivot_2 = obj["pivot_2"]
#link3 = pivot_2["link3"]

#link1.set_object(geom.Box([0.01, 0.01, 0.3]))
#link2.set_object(geom.Box([0.01, 0.01, 0.3]))
#link3.set_object(geom.Box([0.01, 0.01, 0.3]))

#link1.set_transform(tf.translation_matrix([0.0,0.0,0]))
#link2.set_transform(tf.translation_matrix([0.0,0.0,0.3]))
#link3.set_transform(tf.translation_matrix([0.0, 0.0, 0.6]))



In [ ]:
for t in range(1000):
    q_ddot = lagrange(q, q_dot)
    q, q_dot = euler(dt, q, q_dot, q_ddot) 
    
    rotation = tf.rotation_matrix(q, [0, 1, 0])
    translation = tf.translation_matrix([0.0, 0.0, (_l1 / 2)])
    final_transform = np.dot(rotation, translation)

    vis["pendulum_link"].set_transform(final_transform)

    time.sleep(dt)